In [26]:
import datasets
import os
import openai
import numpy as np
with open(os.path.expanduser('~/.openai_api_key'), 'r') as file:
    openai.api_key = file.read().replace('\n', '')

import adatest

In [4]:
d = datasets.load_dataset('bigbench', 'anachronisms')

Generating default split:   0%|          | 0/230 [00:00<?, ? examples/s]

2022-09-30 09:09:23.645979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/opencv-3.1.0/lib:/usr/local/cuda-10.0/lib64:/usr/lib/x86_64-linux-gnu/openmpi/lib/
2022-09-30 09:09:23.646048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/opencv-3.1.0/lib:/usr/local/cuda-10.0/lib64:/usr/lib/x86_64-linux-gnu/openmpi/lib/
2022-09-30 09:09:23.646089: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/opencv-3.1.0/lib

Generating train split:   0%|          | 0/184 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/46 [00:00<?, ? examples/s]

Dataset bigbench downloaded and prepared to /home/marcotcr/.cache/huggingface/datasets/bigbench/anachronisms/1.0.0/7d2f6e537fa937dfaac8b1c1df782f2055071d3fd8e4f4ae93d28012a354ced0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
import numpy as np
inputs = d['validation']['inputs']
inputs = [x.split('\n')[0] for x in inputs]
labels = np.array([int(x[0] == 'Yes') for x in d['validation']['targets']])

In [86]:
# d['validation']['inputs']

In [30]:
class OpenAIModel(adatest.Model):
    def __init__(self, model="text-davinci-002-", quote="\"", temperature=0.7, top_p=1, max_length=30, n=1):
        self.model = model
        self.api_key = openai.api_key
        self.quote = quote
        self.temperature = temperature
        self.top_p = top_p
        self.max_length = max_length
        self.n = n
    def __call__(self, strings):
        resp = openai.Completion.create(
            model=self.model,
            prompt=strings,
            max_tokens=self.max_length,
            temperature=self.temperature,
            top_p=self.top_p,
            n=self.n,
            stop=self.quote,
        )
        return [x["text"] for x in resp['choices']]

gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='', n=1)


In [32]:
def anachronism(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence and the time periods of each entity in it, tell me if it could have happened or not.
Sentence: I wrote about shakespeare
Entities and dates:
I -> 21st century
Shakespeare -> 16th century
Could the sentence be true based on the dates alone: Yes
----
Sentence: Shakespeare wrote about me

Entities and dates:
Shakespeare -> 16th century
I -> 21st century

Could the sentence be true based on the dates alone: No
----
Sentence: %s''' % x
    return gpt3(prompt)


In [69]:
def anachronismv2(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence tell me if it could have happened or not based on the time periods of the entities.
Sentence: I wrote about shakespeare
Could the sentence be true based on the time periods alone: Yes
----
Sentence: Shakespeare wrote about me
Could the sentence be true based on the time periods alone: No
----
Sentence: %s''' % x
    return gpt3(prompt)

In [78]:
def anachronismv3(x):
    prompt = '''Given a sentence, extract the entities and their relationships, and tell me if they are anachronistic.
Sentence: I wrote about shakespeare
Relationships:
- (I, write about, shakespeare) -> not anachronistic, 
----
Sentence: Shakespeare wrote about me
Relationships:
- (shakespeare, write about, me) -> anachronistic
----
Sentence: The builders of the pyramids at Giza listened to jazz during their break.
Relationships:
- (builders, listen to, jazz) -> anachronistic
----
Sentence: %s
''' % x
    return gpt3(prompt)

In [41]:
answers = []
for x in inputs:
    answers.append(anachronism(x))

In [70]:
answers2 = []
for x in inputs:
    answers2.append(anachronismv2(x))

In [79]:

answers3 = []
for x in inputs:
    answers3.append(anachronismv3(x))

In [56]:
preds = np.array([int(x[0].endswith('No')) for x in answers])

In [71]:
preds2 = np.array([int(x[0].endswith('No')) for x in answers2])

In [83]:
preds3 = np.array([1 - int('not anachronistic' in x[0]) for x in answers3])

In [68]:
(preds == labels).mean()

0.7391304347826086

In [72]:
(preds2 == labels).mean()

0.6956521739130435

In [85]:
(preds3 == labels).mean()

0.6304347826086957

In [67]:
for x in np.where(preds != labels)[0]:
    print(inputs[x])
    print(answers[x][0])
    print(labels[x])
    print('----')

Jason, the first person to explore the ancient pyramid in over 2000 years, opened a sealed box inside and found a floppy disk.


Jason -> 21st century
ancient pyramid -> ?

Could the sentence be true based on the dates alone:

The sentence could be true if the ancient pyramid is from the 21st century.
1
----
The first ever movie in color depicted the life of Sacagawea.


Entities and dates:
The first ever movie in color -> late 19th century
Sacagawea -> late 18th century - early 19th century

Could the sentence be true based on the dates alone: No
0
----
Jason, while exploring an ancient pyramid, opened a sealed box inside and found a floppy disk.


Entities and dates:
Jason -> 21st century
floppy disk -> 1970s-2000s

Could the sentence be true based on the dates alone: No
0
----
During their meetings in Bali, George Washington and the delegate of the Tokugawa shogunate exchanged gifts.


Entities and dates:
George Washington -> 18th century
Tokugawa shogunate -> 17th century

Could th